In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import model_selection
import matplotlib.pyplot as plt 
from sklearn.linear_model import RidgeCV,LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR


In [2]:
df = pd.read_csv('Hitters.csv')
df = df.dropna()
dms = pd.get_dummies(df[['League','Division','NewLeague']])
y = df['Salary']
X_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')
X = pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
X[['League_N','Division_W','NewLeague_N']] =X[['League_N','Division_W','NewLeague_N']].astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 train_size=0.25,
                                                 random_state=42)

In [3]:
X_train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
112,490.0,125.0,24.0,81.0,105.0,62.0,13.0,6063.0,1646.0,271.0,847.0,999.0,680.0,869.0,62.0,8.0,1,0,1
293,559.0,141.0,2.0,48.0,61.0,73.0,8.0,3162.0,874.0,16.0,421.0,349.0,359.0,352.0,414.0,9.0,1,0,1
300,510.0,147.0,10.0,56.0,52.0,53.0,7.0,2872.0,821.0,63.0,307.0,340.0,174.0,810.0,99.0,18.0,1,0,1
119,327.0,85.0,3.0,30.0,44.0,20.0,8.0,2140.0,568.0,16.0,216.0,208.0,93.0,91.0,185.0,12.0,0,0,0
9,401.0,92.0,17.0,49.0,66.0,65.0,13.0,5206.0,1332.0,253.0,784.0,890.0,866.0,0.0,0.0,0.0,0,0,0


In [4]:
X_test.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
148,497.0,136.0,7.0,58.0,38.0,26.0,11.0,3871.0,1066.0,40.0,450.0,367.0,241.0,304.0,347.0,10.0,0,0,0
154,537.0,147.0,23.0,58.0,88.0,47.0,10.0,2744.0,730.0,97.0,302.0,351.0,174.0,92.0,257.0,20.0,1,0,1
318,492.0,136.0,5.0,76.0,50.0,94.0,12.0,5511.0,1511.0,39.0,897.0,451.0,875.0,313.0,381.0,20.0,0,0,0
279,439.0,96.0,0.0,44.0,36.0,65.0,4.0,711.0,148.0,1.0,68.0,56.0,99.0,229.0,406.0,22.0,1,0,1
88,155.0,41.0,12.0,21.0,29.0,22.0,16.0,5409.0,1338.0,181.0,746.0,805.0,875.0,165.0,9.0,1.0,0,1,0


In [5]:
knn_model =KNeighborsRegressor().fit(X_train,y_train)

In [6]:
knn_model.n_neighbors

5

In [7]:
y_pred = knn_model.predict(X_test)

In [8]:
np.sqrt(mean_squared_error(y_test,y_pred))

384.0243194233799

In [9]:
RMSE = []

for i in range(10):
    i +=1
    knn_model = KNeighborsRegressor(n_neighbors= i, ).fit(X_train,y_train)
    y_pred = knn_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    RMSE.append(rmse)
    print('i= ',i, 'için RMSE değeri = ',rmse)

i=  1 için RMSE değeri =  398.62374412499224
i=  2 için RMSE değeri =  389.6057708599586
i=  3 için RMSE değeri =  390.4365743318316
i=  4 için RMSE değeri =  384.65993292790586
i=  5 için RMSE değeri =  384.0243194233799
i=  6 için RMSE değeri =  389.9289905453633
i=  7 için RMSE değeri =  381.37730313447514
i=  8 için RMSE değeri =  379.95638674046273
i=  9 için RMSE değeri =  383.18218932081646
i=  10 için RMSE değeri =  383.40515774630154


In [10]:
knn_params = {'n_neighbors':np.arange(1,30,1)}
knn = KNeighborsRegressor()
knn_cv_model = GridSearchCV(knn,knn_params,cv=10).fit(X_train,y_train)

In [11]:
knn_cv_model.best_params_

{'n_neighbors': 10}

In [12]:
knn_tuned = KNeighborsRegressor(n_neighbors=knn_cv_model.best_params_['n_neighbors']).fit(X_train,y_train)

In [13]:
y_pred = knn_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

383.40515774630154